# ALTEGRAD Challenge - Classification

*Abderrahim AIT-AZZI, Sébastien OHLEYER, Mickael SUTTON*

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Performing grid search
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime
from log import _check_log_directory,_initialise_model_log

/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data_dir = "./data"
log_dir = './log'

### Load data

In [3]:
# Glove features
features_train = pd.read_csv(os.path.join(data_dir,'train_features_glove.csv'), sep=',', encoding='latin-1')
features_test = pd.read_csv(os.path.join(data_dir,'test_features_glove.csv'), sep=',', encoding='latin-1')
features_train= features_train.drop(['question1', 'question2'], axis=1)
features_test = features_test.drop(['id','qid1','qid2','question1', 'question2'], axis=1)
data_train = pd.read_csv('./data/train.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2","is_duplicate"])
Y_train=data_train["is_duplicate"].values

# Pagerank features
pagerank_feats_train = pd.read_csv(os.path.join(data_dir,"train_pagerank.csv"), sep=',')
pagerank_feats_test = pd.read_csv(os.path.join(data_dir,"test_pagerank.csv"), sep=',')

# Question frequency
train_question_freq = pd.read_csv(os.path.join(data_dir,'train_question_freq.csv'), sep=',', index_col=0)
test_question_freq = pd.read_csv(os.path.join(data_dir,'test_question_freq.csv'), sep=',', index_col=0)

# Intersection of questions
train_question_inter= pd.read_csv(os.path.join(data_dir,'train_question_inter.csv'), sep=',', index_col=0)
test_question_inter = pd.read_csv(os.path.join(data_dir,'test_question_inter.csv'), sep=',', index_col=0)

# K-cores
train_kcores = pd.read_csv(os.path.join(data_dir,'train_kcores.csv'), sep=',', index_col=0)
test_kcores = pd.read_csv(os.path.join(data_dir,'test_kcores.csv'), sep=',', index_col=0)

# question K-cores
train_question_kcores = pd.read_csv(os.path.join(data_dir,'train_question_kcores.csv'), sep=',', index_col=0)
test_question_kcores = pd.read_csv(os.path.join(data_dir,'test_question_kcores.csv'), sep=',', index_col=0)

# TF-IDF
train_tfidf = pd.read_csv(os.path.join(data_dir,'train_tfidf.csv'), sep=',', index_col=0)
test_tfidf = pd.read_csv(os.path.join(data_dir,'test_tfidf.csv'), sep=',', index_col=0)

# Graph features
train_graph_feat = pd.read_csv(os.path.join(data_dir,'train_graph_feat.csv'), sep=',', index_col=0)
test_graph_feat = pd.read_csv(os.path.join(data_dir,'test_graph_feat.csv'), sep=',', index_col=0)

# Bigram feature
train_bigram_feat = pd.read_csv(os.path.join(data_dir,'train_bigram_feat.csv'), sep=',', index_col=0)
test_bigram_feat = pd.read_csv(os.path.join(data_dir,'test_bigram_feat.csv'), sep=',', index_col=0)

In [4]:
#train_question_kcores.dtypes

### Add features

In [5]:
# Add Pagerank features
features_train[["q1_pr","q2_pr"]]=pagerank_feats_train[["q1_pr","q2_pr"]]
features_test[["q1_pr","q2_pr"]]=pagerank_feats_test[["q1_pr","q2_pr"]]

# Add question frequency features
features_train[["q1_hash","q2_hash","q1_freq","q2_freq"]]=train_question_freq[["q1_hash","q2_hash","q1_freq","q2_freq"]]
features_test[["q1_hash","q2_hash","q1_freq","q2_freq"]]=test_question_freq[["q1_hash","q2_hash","q1_freq","q2_freq"]]

# Add intersection of questions features
features_train['q1_q2_intersect']=train_question_inter['q1_q2_intersect']
features_test['q1_q2_intersect']=test_question_inter['q1_q2_intersect']

# Add K-cores
features_train[["core1","core2","core3"]] = train_kcores[["core1","core2","core3"]]
features_test[["core1","core2","core3"]] = test_kcores[["core1","core2","core3"]]

# Add question K-cores features
features_train[['q1_kcores', 'q2_kcores', 'q1_q2_kcores_ratio', 'q1_q2_kcores_diff', 
                'q1_q2_kcores_diff_normed']]=train_question_kcores[['q1_kcores', 'q2_kcores', 'q1_q2_kcores_ratio', 'q1_q2_kcores_diff', 'q1_q2_kcores_diff_normed']]
features_test[['q1_kcores', 'q2_kcores', 'q1_q2_kcores_ratio', 'q1_q2_kcores_diff', 
               'q1_q2_kcores_diff_normed']]=test_question_kcores[['q1_kcores', 'q2_kcores', 'q1_q2_kcores_ratio', 'q1_q2_kcores_diff', 'q1_q2_kcores_diff_normed']]

# Add TF-IDF features
features_train[['word_match','tfidf_wm','tfidf_wm_stops','jaccard','wc_diff','wc_ratio','wc_diff_unique','wc_ratio_unique','wc_diff_unq_stop','wc_ratio_unique_stop','same_start',
 'char_diff','char_diff_unq_stop','total_unique_words','total_unq_words_stop','char_ratio']]=train_tfidf[['word_match','tfidf_wm','tfidf_wm_stops','jaccard','wc_diff','wc_ratio','wc_diff_unique','wc_ratio_unique','wc_diff_unq_stop','wc_ratio_unique_stop','same_start',
 'char_diff','char_diff_unq_stop','total_unique_words','total_unq_words_stop','char_ratio']]
features_test[['word_match','tfidf_wm','tfidf_wm_stops','jaccard','wc_diff','wc_ratio','wc_diff_unique','wc_ratio_unique','wc_diff_unq_stop','wc_ratio_unique_stop','same_start',
 'char_diff','char_diff_unq_stop','total_unique_words','total_unq_words_stop','char_ratio']]=test_tfidf[['word_match','tfidf_wm','tfidf_wm_stops','jaccard','wc_diff','wc_ratio','wc_diff_unique','wc_ratio_unique','wc_diff_unq_stop','wc_ratio_unique_stop','same_start',
 'char_diff','char_diff_unq_stop','total_unique_words','total_unq_words_stop','char_ratio']]

# Add graph features
features_train[['q1_neigh','q2_neigh','common_neigh', 'distinct_neigh', 'clique_size']] = train_graph_feat[['q1_neigh','q2_neigh','common_neigh', 'distinct_neigh', 'clique_size']]
features_test[['q1_neigh','q2_neigh','common_neigh', 'distinct_neigh', 'clique_size']] = test_graph_feat[['q1_neigh','q2_neigh','common_neigh', 'distinct_neigh', 'clique_size']]

# Add bigram features
features_train[['bigram_coocurence','bigram_distinct','bigram_nostpwrd_coocurence', 'bigram_nostpwrd_coocurence']] = train_bigram_feat[['bigram_coocurence','bigram_distinct','bigram_nostpwrd_coocurence', 'bigram_nostpwrd_coocurence']]
features_test[['bigram_coocurence','bigram_distinct','bigram_nostpwrd_coocurence', 'bigram_nostpwrd_coocurence']] = test_bigram_feat[['bigram_coocurence','bigram_distinct','bigram_nostpwrd_coocurence', 'bigram_nostpwrd_coocurence']]


### Create train and test matrices

In [6]:
X_train= features_train
X_test = features_test
X_train=X_train.replace([np.inf, -np.inf], np.nan)
X_train=X_train.fillna(value=0)
X_test=X_test.replace([np.inf, -np.inf], np.nan)
X_test=X_test.fillna(value=0)

In [7]:
X_train.columns

Index(['is_duplicate', 'len_q1', 'len_q2', 'diff_len', 'len_char_q1',
       'len_char_q2', 'len_word_q1', 'len_word_q2', 'common_words',
       'fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio',
       'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
       'fuzz_token_set_ratio', 'fuzz_token_sort_ratio', 'wmd', 'norm_wmd',
       'cosine_distance', 'cityblock_distance', 'jaccard_distance',
       'canberra_distance', 'euclidean_distance', 'minkowski_distance',
       'braycurtis_distance', 'skew_q1vec', 'skew_q2vec', 'kur_q1vec',
       'kur_q2vec', 'q1_pr', 'q2_pr', 'q1_hash', 'q2_hash', 'q1_freq',
       'q2_freq', 'q1_q2_intersect', 'core1', 'core2', 'core3', 'q1_kcores',
       'q2_kcores', 'q1_q2_kcores_ratio', 'q1_q2_kcores_diff',
       'q1_q2_kcores_diff_normed', 'word_match', 'tfidf_wm', 'tfidf_wm_stops',
       'jaccard', 'wc_diff', 'wc_ratio', 'wc_diff_unique', 'wc_ratio_unique',
       'wc_diff_unq_stop', 'wc_ratio_unique_stop', 'same_start', 'char_diff',

# Classifier

### A. Feature selection

In [ ]:
#initialize model directory
log_name = (datetime.now().strftime('%d-%m-%Y_%H-%M-%S'))
_check_log_directory(os.path.join(log_dir,log_name))

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in X_train.columns if x not in ['is_duplicate']]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

cv_folds=5
early_stopping_rounds=50
# modelfit(xgb1, X_train, predictors)
alg=xgb1
dtrain=X_train.copy()
xgb_param = alg.get_xgb_params()
xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain['is_duplicate'].values)

In [ ]:
#alg.booster().attributes()

In [ ]:
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
    metrics='auc', early_stopping_rounds=early_stopping_rounds,verbose_eval=True)
alg.set_params(n_estimators=cvresult.shape[0])

In [ ]:
#Fit the algorithm on the data
alg.fit(dtrain[predictors], dtrain['is_duplicate'],eval_metric='auc',verbose=True)

#Predict training set:
dtrain_predictions = alg.predict(dtrain[predictors])
dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

In [ ]:
#Print model report:
print ("\nModel Report")
print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['is_duplicate'].values, dtrain_predictions))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['is_duplicate'], dtrain_predprob))

In [ ]:
feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#feat_imp.plot(kind='bar', title='Feature Importances')
#plt.ylabel('Feature Importance Score')
#plt.show()
print(feat_imp)
#feat_imp.to_csv(os.path.join(log_dir,'feature_important.csv'))

#### Dropping inefficient features

In [ ]:
#X_train = features_train.drop(['is_duplicate','core1','core2','core3','q1_neigh','q2_neigh','len_word_q2','len_word_q1','same_start','wc_diff_unq_stop','wc_diff_unique','q1_freq','wc_diff','q1_kcores','q1_q2_kcores_diff','q1_q2_intersect','q1_q2_kcores_ratio','euclidean_distance','len_word_q2','q2_freq',
#                              'norm_wmd','q1_q2_kcores_diff_normed','common_neigh','q2_kcores','diff_len','len_word_q1','common_words','clique_size','total_unique_words','total_unq_words_stop','wc_ratio_unique_stop','wc_ratio_unique'], axis=1)

In [ ]:
#X_test = features_test.drop(['core1','core2','core3','q1_neigh','q2_neigh','len_word_q2','len_word_q1','same_start','wc_diff_unq_stop','wc_diff_unique','q1_freq','wc_diff','q1_kcores','q1_q2_kcores_diff','q1_q2_intersect','q1_q2_kcores_ratio','euclidean_distance','len_word_q2','q2_freq',
#                              'norm_wmd','q1_q2_kcores_diff_normed','common_neigh','q2_kcores','diff_len','len_word_q1','common_words','clique_size','total_unique_words','total_unq_words_stop','wc_ratio_unique_stop','wc_ratio_unique'], axis=1)

In [ ]:
xgb1.fit(X_train,Y_train)

#### Make prediction on test set

In [ ]:
y_pred1 = xgb1.predict_proba(X_test)
with open("submission_file.csv", 'w') as f:
    f.write("Id,Score\n")
    for i in range(y_pred1.shape[0]):
        f.write(str(i)+','+str(y_pred1[i][1])+'\n')

## B. Cross val

### B.1 light gb

In [8]:
from sklearn.model_selection import StratifiedKFold
NUM_FOLDS = 5
RANDOM_SEED = 2017
np.random.seed(RANDOM_SEED)
import lightgbm as lgb

In [9]:
print('Number of features on train matrix: ',len(X_train.columns))
print('Number of features on test matrix: ',len(X_test.columns))

Number of features on train matrix:  65
Number of features on test matrix:  64


In [10]:
#initialize model directory
log_name = (datetime.now().strftime('%d-%m-%Y_%H-%M-%S'))
log_filepath = os.path.join(log_dir,log_name,'lighgb.csv')
_check_log_directory(os.path.join(log_dir,log_name))
_initialise_model_log(log_filepath)

Attempting to make log directory at ./log/27-01-2018_18-22-28


In [34]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [35]:
y_test_pred = np.zeros((len(X_test), NUM_FOLDS))
cv_val_scores = []
cv_train_scores = []

X_train_values=X_train.values
X_test_values=X_test.values
for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_values, Y_train)):
    print('Fitting fold {fold_num + 1} of {kfold.n_splits}')
    
    print(len(ix_train))
    print(X_train_values.shape)
    X_fold_train = X_train_values[ix_train,:]
    X_fold_val = X_train_values[ix_val,:]

    y_fold_train = Y_train[ix_train]
    y_fold_val = Y_train[ix_val]
    
    lgb_params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'device': 'cpu',
        'feature_fraction': 0.486,
        'num_leaves': 170,
        'lambda_l2': 50,
        'learning_rate': 0.01,
        'num_boost_round': 5000,
        'early_stopping_rounds': 10,
        'verbose': 1,
        'bagging_fraction_seed': RANDOM_SEED,
        'feature_fraction_seed': RANDOM_SEED,
    }
    
    lgb_data_train = lgb.Dataset(X_fold_train, y_fold_train)
    lgb_data_val = lgb.Dataset(X_fold_val, y_fold_val)    
    evals_result = {}
    
    model = lgb.train(
        lgb_params,
        lgb_data_train,
        valid_sets=[lgb_data_train, lgb_data_val],
        evals_result=evals_result,
        num_boost_round=lgb_params['num_boost_round'],
        early_stopping_rounds=lgb_params['early_stopping_rounds'],
        verbose_eval=False,
    )
    
    fold_train_scores = evals_result['training'][lgb_params['metric']]
    fold_val_scores = evals_result['valid_1'][lgb_params['metric']]
    
    print('Fold {}: {} rounds, training loss {:.6f}, validation loss {:.6f}'.format(
        fold_num + 1,
        len(fold_train_scores),
        fold_train_scores[-1],
        fold_val_scores[-1],
    ))
    print()
    cv_train_scores.append(fold_train_scores[-1])
    cv_val_scores.append(fold_val_scores[-1])
    y_test_pred[:, fold_num] = model.predict(X_test).reshape(-1)

Fitting fold {fold_num + 1} of {kfold.n_splits}
64079
(80100, 65)


/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/lightgbm/engine.py:103: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fold 1: 5000 rounds, training loss 0.000089, validation loss 0.000091

Fitting fold {fold_num + 1} of {kfold.n_splits}
64080
(80100, 65)
Fold 2: 5000 rounds, training loss 0.000089, validation loss 0.000096

Fitting fold {fold_num + 1} of {kfold.n_splits}
64080
(80100, 65)
Fold 3: 5000 rounds, training loss 0.000089, validation loss 0.000099

Fitting fold {fold_num + 1} of {kfold.n_splits}
64080
(80100, 65)
Fold 4: 5000 rounds, training loss 0.000089, validation loss 0.000095

Fitting fold {fold_num + 1} of {kfold.n_splits}
64081
(80100, 65)
Fold 5: 5000 rounds, training loss 0.000089, validation loss 0.000100



In [31]:
feat_imp = pd.DataFrame({
    'column': list(X_train.columns),
    'importance': model.feature_importance(),
}).sort_values(by='importance')

In [32]:
print('Final CV score:', cv_val_scores)
with open(log_filepath, 'a') as fp:
    a = csv.writer(fp, delimiter=',')
    data = [[datetime.now().strftime('%d-%m-%Y_%H-%M-%S'), 'lightgb', 
             cv_val_scores, np.mean(cv_val_scores), 
             cv_train_scores, np.mean(cv_train_scores),
             lgb_params, feat_imp.to_dict()]]
    a.writerows(data)

Final CV score: [9.08968021240727e-05, 9.57143085722637e-05, 9.926108085904113e-05, 9.501631228094978e-05, 0.00010013330435253106]


In [22]:
y_test = np.mean(y_test_pred, axis=1)
with open("submission_file.csv", 'w') as f:
    f.write("Id,Score\n")
    for i in range(y_test.shape[0]):
        f.write(str(i)+','+str(y_test[i])+'\n')

### B.2 Xgboost

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
import xgboost as xgb

X_train, X_valid, y_train, y_valid = train_test_split(X_train1, Y_train, test_size=0.1, random_state=4242)

# UPDownSampling
pos_train = X_train[y_train == 1]
neg_train = X_train[y_train == 0]
X_train = pd.concat((neg_train, pos_train.iloc[:int(0.8 * len(pos_train))], neg_train))
y_train = np.array(
    [0] * neg_train.shape[0] + [1] * pos_train.iloc[:int(0.8 * len(pos_train))].shape[0] + [0] * neg_train.shape[0])
print(np.mean(y_train))
del pos_train, neg_train

pos_valid = X_valid[y_valid == 1]
neg_valid = X_valid[y_valid == 0]
X_valid = pd.concat((neg_valid, pos_valid.iloc[:int(0.8 * len(pos_valid))], neg_valid))
y_valid = np.array(
    [0] * neg_valid.shape[0] + [1] * pos_valid.iloc[:int(0.8 * len(pos_valid))].shape[0] + [0] * neg_valid.shape[0])
print(np.mean(y_valid))
del pos_valid, neg_valid

params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 7
params['subsample'] = 0.6
params['base_score'] = 0.2
# params['scale_pos_weight'] = 0.2

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 2500, watchlist, early_stopping_rounds=50, verbose_eval=50)
print(log_loss(y_valid, bst.predict(d_valid)))
d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test)

In [ ]:
os.path.join(log_dir,'test','train','2')